# 🧩 **Módulo 7: RAG Completo - Juntando Todos os Pedaços**

> *Agora vamos montar o quebra-cabeça completo e ver a mágica acontecer!*

---

## **Aula 7.1: O que é RAG Completo e por que é como um Chef Completo?**

---

### **Tá, mas o que é RAG Completo?**

RAG Completo é como um **chef experiente** que pega todos os ingredientes (documentos), prepara eles (embeddings), organiza na despensa (vector store), busca o que precisa (retrieval) e cozinha um prato delicioso (resposta)! 👨‍🍳

**🖼️ Sugestão de imagem**: Um chef cozinhando com todos os ingredientes organizados

**Por que RAG Completo é importante?**

É como a diferença entre:
- �� **Ingredientes soltos**: Difícil de usar
- 🍽️ **Prato completo**: Delicioso e funcional

### **Analogia do Dia a Dia**

RAG Completo é como **montar um quebra-cabeça gigante**:
- 🧩 **Peças soltas**: Documentos, embeddings, vector stores
- ��️ **Imagem completa**: Sistema RAG funcionando perfeitamente
- �� **Resultado**: Respostas precisas e confiáveis

**Sem RAG Completo** seria como ter todas as peças do quebra-cabeça mas nunca montá-lo! 😅

---

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou o notebook `00_setup_colab.ipynb` primeiro, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0
!pip install matplotlib>=3.5.0
!pip install scikit-learn>=1.3.0

print("✅ Dependências instaladas com sucesso!")
print("�� Pronto para montar o RAG completo!")

In [ ]:
# �� IMPORTAÇÕES PARA RAG COMPLETO
import os
from dotenv import load_dotenv

# LangChain
from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

# Utilitários
import numpy as np
import pandas as pd

print("✅ Bibliotecas importadas com sucesso!")
print("🧩 Pronto para criar o RAG completo!")

### **Exemplo Prático - Criando Nosso Primeiro RAG Completo**

Vamos criar um sistema RAG completo sobre tecnologia. É como montar um quebra-cabeça sobre o futuro! 🚀

In [ ]:
# �� EXEMPLO PRÁTICO: CRIANDO NOSSO PRIMEIRO RAG COMPLETO

# Criando base de conhecimento sobre tecnologia
base_conhecimento_tech = [
    "Inteligência Artificial (IA) é um campo da computação que busca criar sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. A IA combina ciência da computação, matemática, psicologia e outras disciplinas.",
    
    "Machine Learning é um subcampo da IA que permite aos computadores aprender sem serem explicitamente programados. Os algoritmos identificam padrões nos dados para fazer previsões ou tomar decisões.",
    
    "Deep Learning é um subcampo do Machine Learning que usa redes neurais com múltiplas camadas para aprender representações complexas dos dados. Requer grandes quantidades de dados e poder computacional significativo.",
    
    "RAG (Retrieval Augmented Generation) é uma técnica que combina busca de informações com geração de texto. Em vez de gerar respostas baseadas apenas no conhecimento prévio, o RAG consulta documentos relevantes antes de responder.",
    
    "ChatGPT é um modelo de linguagem desenvolvido pela OpenAI. É baseado na arquitetura GPT (Generative Pre-trained Transformer) e é capaz de gerar texto humano de alta qualidade.",
    
    "Blockchain é uma tecnologia de registro distribuído que permite transações seguras e transparentes sem a necessidade de intermediários. É a base das criptomoedas como Bitcoin.",
    
    "Cloud Computing é a entrega de serviços de computação pela internet. Inclui servidores, armazenamento, bancos de dados, redes, software e inteligência artificial.",
    
    "Internet das Coisas (IoT) é a rede de dispositivos físicos conectados à internet. Esses dispositivos coletam e trocam dados, permitindo automação e monitoramento inteligente.",
    
    "Realidade Virtual (VR) é uma tecnologia que cria ambientes simulados em 3D. Os usuários podem interagir com esses ambientes usando dispositivos especiais como óculos VR.",
    
    "Realidade Aumentada (AR) é uma tecnologia que sobrepõe informações digitais ao mundo real. Diferente da VR, a AR não substitui o ambiente real, mas o complementa.",
    
    "5G é a quinta geração de tecnologia de rede móvel. Oferece velocidades muito mais rápidas, menor latência e maior capacidade de conexão simultânea de dispositivos.",
    
    "Cibersegurança é a proteção de sistemas, redes e programas contra ataques digitais. Inclui medidas para proteger dados, prevenir ataques e garantir a privacidade dos usuários.",
    
    "Big Data refere-se a conjuntos de dados muito grandes e complexos que não podem ser processados por métodos tradicionais. Requer técnicas especiais de análise e processamento.",
    
    "Automação é o uso de tecnologia para executar tarefas sem intervenção humana. Pode incluir robótica, software e sistemas inteligentes que substituem ou complementam o trabalho humano.",
    
    "Computação Quântica é um tipo de computação que usa fenômenos quânticos como superposição e entrelaçamento. Tem potencial para resolver problemas que são impossíveis para computadores clássicos."
]

print(f"📚 Base de conhecimento criada com {len(base_conhecimento_tech)} documentos sobre tecnologia")
print("�� Pronto para montar nosso RAG completo!")

## **Aula 7.2: Passo 1 - Preparando os Ingredientes (Document Loaders e Text Splitting)**

---

### **Tá, mas como preparar os ingredientes?**

Primeiro, vamos preparar nossos "ingredientes" - transformar os documentos em pedaços gerenciáveis! 🥕

**🖼️ Sugestão de imagem**: Um chef cortando e preparando ingredientes

In [ ]:
# �� PASSO 1: PREPARANDO OS INGREDIENTES

# Criando documentos do LangChain
documentos = [Document(page_content=texto, metadata={"fonte": "base_tech", "id": i}) 
              for i, texto in enumerate(base_conhecimento_tech)]

print(f"📄 {len(documentos)} documentos criados")

# Dividindo em pedaços menores (Text Splitting)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # Tamanho de cada pedaço
    chunk_overlap=50,    # Sobreposição entre pedaços
    separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
)

# Aplicando o splitter
textos_divididos = text_splitter.split_documents(documentos)

print(f"🔪 Documentos divididos em {len(textos_divididos)} pedaços")
print(f"📏 Tamanho médio: {sum(len(doc.page_content) for doc in textos_divididos) / len(textos_divididos):.0f} caracteres")

# Mostrando alguns pedaços
print("\n🍽️ Primeiros 3 pedaços preparados:")
for i, doc in enumerate(textos_divididos[:3], 1):
    print(f"\n--- Pedaço {i} ---")
    print(doc.page_content[:150] + "..." if len(doc.page_content) > 150 else doc.page_content)

print("\n✅ Ingredientes preparados com sucesso!")

## **Aula 7.3: Passo 2 - Criando os Embeddings (Transformando em Números)**

---

### **Tá, mas como transformar texto em números?**

Agora vamos transformar nossos textos em **coordenadas numéricas** que o computador entende! 🧮

**🖼️ Sugestão de imagem**: Palavras sendo transformadas em coordenadas no espaço

In [ ]:
# 🧮 PASSO 2: CRIANDO OS EMBEDDINGS

# Criando modelo de embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("🧠 Modelo de embeddings carregado!")
print(f"📐 Dimensão dos embeddings: {embeddings.client.get_sentence_embedding_dimension()}")

# Testando embeddings com algumas frases
frases_teste = [
    "Inteligência Artificial é incrível",
    "Machine Learning é um subcampo da IA",
    "O tempo está chuvoso hoje"
]

print("\n🧪 Testando embeddings:")
for i, frase in enumerate(frases_teste, 1):
    embedding = embeddings.embed_query(frase)
    print(f"   {i}. '{frase}' -> {len(embedding)} números")

print("\n✅ Embeddings funcionando perfeitamente!")

## **Aula 7.4: Passo 3 - Organizando na Despensa (Vector Store)**

---

### **Tá, mas onde guardar todos esses números?**

Agora vamos organizar nossos embeddings em uma **"despensa inteligente"** onde é fácil encontrar o que precisamos! ��️

**🖼️ Sugestão de imagem**: Uma despensa organizada com ingredientes bem arrumados

In [ ]:
# 🗄️ PASSO 3: ORGANIZANDO NA DESPENSA (VECTOR STORE)

# Criando vector store com Chroma
vectorstore = Chroma.from_documents(
    documents=textos_divididos,
    embedding=embeddings,
    collection_name="base_tecnologia"
)

print("🗄️ Vector store criado com sucesso!")
print(f"📚 {len(textos_divididos)} documentos indexados")
print(f"�� Embeddings organizados por similaridade")

# Testando a busca no vector store
print("\n🔍 Testando busca no vector store:")
pergunta_teste = "O que é inteligência artificial?"
docs_encontrados = vectorstore.similarity_search(pergunta_teste, k=2)

print(f"❓ Pergunta: {pergunta_teste}")
print("📄 Documentos encontrados:")
for i, doc in enumerate(docs_encontrados, 1):
    print(f"   {i}. {doc.page_content[:100]}...")

print("\n✅ Despensa organizada e funcionando!")

## **Aula 7.5: Passo 4 - Configurando o Chef (LLM)**

---

### **Tá, mas quem vai cozinhar a resposta?**

Agora vamos configurar nosso **"chef inteligente"** que vai pegar os ingredientes e criar a resposta final! 👨‍🍳

**🖼️ Sugestão de imagem**: Um chef cozinhando com ingredientes frescos

In [ ]:
# ��‍🍳 PASSO 4: CONFIGURANDO O CHEF (LLM)

# Configurando o LLM (nosso chef)
def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI (se tiver API key)
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except:
        pass
    
    # Tentativa 2: Hugging Face (gratuito)
    try:
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except:
        pass
    
    # Fallback: LLM simples
    return HuggingFaceHub(
        repo_id="google/flan-t5-base",
        model_kwargs={"temperature": 0.7, "max_length": 512}
    )

# Configurando o LLM
llm = get_llm_colab()

print(f"👨‍🍳 Chef configurado: {type(llm).__name__}")
print("�� Pronto para cozinhar respostas deliciosas!")

## **Aula 7.6: Passo 5 - Montando o RAG Completo**

---

### **Tá, mas como juntar tudo?**

Agora vamos **montar o quebra-cabeça completo** - juntar todos os pedaços para criar nosso sistema RAG! 🧩

**🖼️ Sugestão de imagem**: Um quebra-cabeça sendo montado peça por peça

In [ ]:
# 🧩 PASSO 5: MONTANDO O RAG COMPLETO

# Criando o sistema RAG completo
rag_system = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=True
)

print("🧩 Sistema RAG completo montado!")
print("🎯 Pronto para responder perguntas sobre tecnologia!")

### **Teste Rápido - Nosso RAG em Ação!**

Vamos testar nosso sistema RAG completo com algumas perguntas:

In [ ]:
# 🧪 TESTE RÁPIDO: NOSSO RAG EM AÇÃO!

# Perguntas para testar o sistema
perguntas_teste = [
    "O que é inteligência artificial?",
    "Como funciona o machine learning?",
    "Qual a diferença entre VR e AR?",
    "O que é blockchain?",
    "Como funciona o 5G?"
]

print("�� TESTANDO NOSSO RAG COMPLETO")
print("=" * 50)

for i, pergunta in enumerate(perguntas_teste, 1):
    print(f"\n❓ Pergunta {i}: {pergunta}")
    print("-" * 40)
    
    try:
        # Executando o RAG
        resultado = rag_system({"query": pergunta})
        
        print(f"🤖 Resposta: {resultado['result'][:200]}...")
        
        if resultado['source_documents']:
            print(f"�� Fontes consultadas: {len(resultado['source_documents'])} documentos")
        
    except Exception as e:
        print(f"❌ Erro: {e}")
    
    print("-" * 40)

print("\n🎉 Sistema RAG funcionando perfeitamente!")

## **Aula 7.7: RAG Conversacional - Com Memória**

---

### **Tá, mas e se eu quiser conversar com o RAG?**

Vamos criar um RAG **conversacional** que lembra do que você falou antes! É como conversar com um amigo que tem uma biblioteca na cabeça! ��️

**🖼️ Sugestão de imagem**: Duas pessoas conversando, uma com uma biblioteca flutuando sobre a cabeça

In [ ]:
# 🗣️ RAG CONVERSACIONAL - COM MEMÓRIA

# Criando memória para o RAG
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Criando RAG conversacional
rag_conversacional = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    verbose=True
)

print("🗣️ RAG conversacional criado!")
print("🧠 Com memória para lembrar da conversa!")

In [ ]:
# 🧪 TESTANDO RAG CONVERSACIONAL

print("��️ TESTANDO RAG CONVERSACIONAL")
print("=" * 50)

# Simulando uma conversa
conversa = [
    "O que é inteligência artificial?",
    "E como ela se relaciona com machine learning?",
    "Você pode me dar exemplos práticos?",
    "Lembra do que eu perguntei sobre IA?"
]

for i, mensagem in enumerate(conversa, 1):
    print(f"\n�� Usuário {i}: {mensagem}")
    print("-" * 40)
    
    try:
        # Executando RAG conversacional
        resultado = rag_conversacional({"question": mensagem})
        
        print(f"🤖 RAG: {resultado['answer'][:200]}...")
        
        if resultado['source_documents']:
            print(f"📄 Consultou {len(resultado['source_documents'])} documentos")
        
    except Exception as e:
        print(f"❌ Erro: {e}")
    
    print("-" * 40)

print("\n�� RAG conversacional funcionando perfeitamente!")

### **Desafio do Módulo - Criando um RAG Personalizado**

Vamos criar um RAG personalizado que pode ser adaptado para diferentes domínios:

In [ ]:
# 🎯 DESAFIO: CRIANDO UM RAG PERSONALIZADO

class RAGPersonalizado:
    """Sistema RAG personalizado e flexível"""
    
    def __init__(self, nome_dominio="tecnologia"):
        self.nome_dominio = nome_dominio
        self.vectorstore = None
        self.llm = None
        self.rag_system = None
        
    def configurar_llm(self):
        """Configura o LLM"""
        self.llm = get_llm_colab()
        print(f"👨‍🍳 Chef configurado para {self.nome_dominio}")
        
    def carregar_documentos(self, documentos):
        """Carrega e processa documentos"""
        # Criando documentos do LangChain
        docs = [Document(page_content=texto, metadata={"dominio": self.nome_dominio, "id": i}) 
                for i, texto in enumerate(documentos)]
        
        # Dividindo em pedaços
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=50
        )
        
        textos_divididos = text_splitter.split_documents(docs)
        
        # Criando embeddings e vector store
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        
        self.vectorstore = Chroma.from_documents(
            documents=textos_divididos,
            embedding=embeddings,
            collection_name=f"base_{self.nome_dominio}"
        )
        
        print(f"📚 {len(textos_divididos)} documentos carregados para {self.nome_dominio}")
        
    def criar_sistema_rag(self):
        """Cria o sistema RAG completo"""
        self.rag_system = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True
        )
        
        print(f"�� Sistema RAG criado para {self.nome_dominio}")
        
    def perguntar(self, pergunta):
        """Faz uma pergunta ao sistema RAG"""
        if not self.rag_system:
            return "Sistema RAG não configurado!"
        
        try:
            resultado = self.rag_system({"query": pergunta})
            return resultado['result']
        except Exception as e:
            return f"Erro: {e}"

# Testando o RAG personalizado
print("🎯 TESTANDO RAG PERSONALIZADO")
print("=" * 50)

# Criando RAG para tecnologia
rag_tech = RAGPersonalizado("tecnologia")
rag_tech.configurar_llm()
rag_tech.carregar_documentos(base_conhecimento_tech)
rag_tech.criar_sistema_rag()

# Testando
pergunta_teste = "O que é deep learning?"
print(f"\n❓ Pergunta: {pergunta_teste}")
resposta = rag_tech.perguntar(pergunta_teste)
print(f"🤖 Resposta: {resposta[:200]}...")

print("\n🎉 RAG personalizado funcionando perfeitamente!")

### **Na Prática, Meu Consagrado!** 💪

**O que construímos no RAG Completo:**

1. ✅ **Sistema RAG Básico** - Funcionando perfeitamente
2. ✅ **RAG Conversacional** - Com memória de conversa
3. ✅ **RAG Personalizado** - Adaptável para qualquer domínio
4. ✅ **Sistema Completo** - Do documento à resposta final

### **Vantagens do RAG Completo**

- **Precisão**: Respostas baseadas em documentos reais
- **Flexibilidade**: Adaptável para qualquer domínio
- **Memória**: Lembra do contexto da conversa
- **Escalabilidade**: Pode lidar com grandes volumes de dados
- **Confiabilidade**: Reduz alucinações da IA

### **Próximos Passos**

**🖼️ Sugestão de imagem**: Fluxo completo do RAG mostrando todos os componentes funcionando juntos

**🎯 Próximo módulo**: Vamos criar **Projetos Práticos** - aplicações reais do RAG!

**�� Resumo do Módulo 7**:
- ✅ Montamos o RAG completo passo a passo
- ✅ Criamos sistemas básicos e conversacionais
- ✅ Desenvolvemos RAG personalizado
- ✅ Testamos com exemplos reais

**🚀 Agora você tem um sistema RAG completo e funcional!**

---

**�� Dica do Pedro**: O RAG é como um quebra-cabeça - cada peça é importante, mas o resultado final é muito maior que a soma das partes!

**🚀 Próximo módulo**: Projetos Práticos - Colocando a mão na massa! 🛠️